In [12]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import os
import time
import traceback # To print detailed errors
import imageio # Do tworzenia GIFów
from tqdm import tqdm # Pasek postępu
from collections import Counter
import re

In [25]:
# --- Konfiguracja ---
OUTPUT_DIR = "tsp_sa_results_acceptance_funcs" # Nowy folder
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Parametry algorytmu Symulowanego Wyżarzania
INITIAL_TEMP = 10000
FINAL_TEMP = 1e-3
FINAL_TEMP_LOG = 50 # Specjalna T_final dla log

COOLING_RATES_EXP = [0.999, 0.99, 0.95] # Ograniczono liczbę dla szybszych testów
COOLING_RATE_LIN = 50
COOLING_RATE_LOG = 25.0
ITERATIONS_PER_TEMP = 1

# Wartości n do przetestowania
N_VALUES = [10, 20, 50] # Ograniczono liczbę dla szybszych testów

# <<< NOWOŚĆ: Typy funkcji akceptacji >>>
ACCEPTANCE_METROPOLIS = 'metropolis'
ACCEPTANCE_LOGISTIC = 'logistic'
ACCEPTANCE_RATIO = 'ratio'
ACCEPTANCE_FUNCTIONS = [ACCEPTANCE_METROPOLIS, ACCEPTANCE_LOGISTIC, ACCEPTANCE_RATIO]

# --- Funkcje pomocnicze ---
# (bez zmian - calculate_distance, calculate_total_distance)
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def calculate_total_distance(points, order):
    total_dist = 0
    num_points = len(points)
    for i in range(num_points):
        current_point_idx = order[i]
        next_point_idx = order[(i + 1) % num_points]
        total_dist += calculate_distance(points[current_point_idx], points[next_point_idx])
    return total_dist

# --- Generatory punktów ---
# (bez zmian - generate_uniform_points, generate_normal_clusters, generate_separated_groups)
def generate_uniform_points(n, x_range=(0, 100), y_range=(0, 100)):
    if n <= 0: return np.array([])
    points = np.random.rand(n, 2)
    points[:, 0] = points[:, 0] * (x_range[1] - x_range[0]) + x_range[0]
    points[:, 1] = points[:, 1] * (y_range[1] - y_range[0]) + y_range[0]
    return points

def generate_normal_clusters(n, num_clusters=4, spread_factor=50, cluster_std_dev=5):
    if n <= 0: return np.array([])
    points = []
    points_per_cluster = n // num_clusters
    if num_clusters <= 0: num_clusters = 1 # Safety check
    centers = np.random.rand(num_clusters, 2) * spread_factor
    for i in range(num_clusters):
        count = points_per_cluster if i < num_clusters - 1 else n - len(points)
        if count <=0 : continue
        cluster_points = np.random.randn(count, 2) * cluster_std_dev + centers[i % len(centers)]
        points.extend(cluster_points)
    while len(points) < n:
         center_idx = random.randrange(len(centers)) if centers.size > 0 else 0
         extra_point = np.random.randn(1, 2) * cluster_std_dev + centers[center_idx]
         points.append(extra_point[0])
    return np.array(points)[:n]

def generate_separated_groups(n, num_groups=9, group_area_size=10, separation_factor=30):
    if n <= 0: return np.array([])
    points = []
    if num_groups <= 0: num_groups = 1
    points_per_group = n // num_groups
    grid_size = math.ceil(math.sqrt(num_groups))
    group_origins = []
    idx = 0
    for r in range(grid_size):
        for c in range(grid_size):
            if idx < num_groups:
                origin_x = c * separation_factor
                origin_y = r * separation_factor
                group_origins.append((origin_x, origin_y))
                idx += 1
    if not group_origins: # Handle case num_groups=0 or calculation failed
         group_origins.append((0,0)) # Default origin

    for i in range(num_groups):
        count = points_per_group if i < num_groups - 1 else n - len(points)
        if count <= 0 and len(points) >= n: break
        if count <= 0: count = 0
        origin = group_origins[i % len(group_origins)]
        group_points = np.random.rand(count, 2) * group_area_size + np.array(origin)
        points.extend(group_points)
    while len(points) < n:
        origin_idx = random.randrange(len(group_origins))
        origin = group_origins[origin_idx]
        extra_point = np.random.rand(1, 2) * group_area_size + np.array(origin)
        points.append(extra_point[0])
    return np.array(points)[:n]

# --- Generatory sąsiadów ---
# (bez zmian - generate_neighbor_consecutive_swap, generate_neighbor_arbitrary_swap)
def generate_neighbor_consecutive_swap(current_order):
    n = len(current_order)
    if n < 2: return current_order[:]
    new_order = current_order[:]
    i = random.randrange(n)
    j = (i + 1) % n
    new_order[i], new_order[j] = new_order[j], new_order[i]
    return new_order

def generate_neighbor_arbitrary_swap(current_order):
    n = len(current_order)
    if n < 2: return current_order[:]
    new_order = current_order[:]
    i, j = random.sample(range(n), 2)
    new_order[i], new_order[j] = new_order[j], new_order[i]
    return new_order

# --- Algorytm Symulowanego Wyżarzania ---
# <<< ZMIANA: Dodano parametr acceptance_func >>>
def simulated_annealing(points, initial_temp, final_temp_default, final_temp_log,
                       cooling_type, cooling_param,
                       acceptance_func, # <<< Nowy parametr
                       iterations_per_temp, neighbor_func):
    n = len(points)
    current_order = list(range(n))
    random.shuffle(current_order)
    current_distance = calculate_total_distance(points, current_order)

    best_order = current_order[:]
    best_distance = current_distance

    temp = initial_temp
    start_temp_for_log = initial_temp

    history_cost = []
    history_best_cost = []
    history_temp = []
    iteration_count = 0
    epoch_count = 0

    if cooling_type == 'log':
        effective_final_temp = final_temp_log
        stop_criterion_info = f"T_final_log={final_temp_log:.1e}"
    else:
        effective_final_temp = final_temp_default
        stop_criterion_info = f"T_final={final_temp_default:.1e}"

    # <<< ZMIANA: Uwzględnij acceptance_func w logu startowym >>>
    print(f"Starting SA: n={n}, T_init={initial_temp:.1f}, {stop_criterion_info}, Cooling={cooling_type}({cooling_param}), Acceptance={acceptance_func}, Neighbor={neighbor_func.__name__}, Iter/Temp={iterations_per_temp}")

    while temp > effective_final_temp and temp > 1e-10:
        history_temp.append(temp)

        for _ in range(iterations_per_temp):
            new_order = neighbor_func(current_order)
            new_distance = calculate_total_distance(points, new_order)
            delta_distance = new_distance - current_distance # delta_distance > 0 means worse

            accept_move = False
            if delta_distance < 0:
                # Zawsze akceptuj lepsze rozwiązania
                accept_move = True
            else:
                # <<< ZMIANA: Oblicz prawdopodobieństwo dla gorszego rozwiązania >>>
                acceptance_probability = 0.0
                if temp > 1e-10: # Unikaj dzielenia przez zero itp.
                    try:
                        if acceptance_func == ACCEPTANCE_METROPOLIS:
                            acceptance_probability = math.exp(-delta_distance / temp)
                        elif acceptance_func == ACCEPTANCE_LOGISTIC:
                             # exp(large_positive) może dać OverflowError
                             exp_arg = delta_distance / temp
                             # Ograniczenie argumentu exp, aby zapobiec OverflowError
                             # Wartość 700 jest bliska granicy dla float64
                             if exp_arg > 700:
                                 acceptance_probability = 0.0 # exp(700) jest ogromne, 1/(1+inf) = 0
                             else:
                                 acceptance_probability = 1.0 / (1.0 + math.exp(exp_arg))
                        elif acceptance_func == ACCEPTANCE_RATIO:
                            # Denominator is temp + delta_distance. Both >= 0, temp > 1e-10.
                            # So denominator is > 1e-10 unless delta_distance is huge negative (impossible here)
                             denominator = temp + delta_distance
                             if denominator > 1e-10: # Should always be true here
                                 acceptance_probability = temp / denominator
                             else: # Safety fallback
                                 acceptance_probability = 0.0

                    except OverflowError:
                        print(f"Warning: OverflowError calculating probability (T={temp:.2e}, dE={delta_distance:.2e}). Prob set to 0.")
                        acceptance_probability = 0.0
                    except ValueError:
                        print(f"Warning: ValueError calculating probability (T={temp:.2e}, dE={delta_distance:.2e}). Prob set to 0.")
                        acceptance_probability = 0.0


                # Podejmij decyzję losową
                if random.random() < acceptance_probability:
                    accept_move = True

            # Zastosuj ruch, jeśli został zaakceptowany
            if accept_move:
                current_order = new_order
                current_distance = new_distance
                # Zaktualizuj najlepsze tylko jeśli *obecne* jest lepsze
                if current_distance < best_distance:
                    best_order = current_order[:]
                    best_distance = current_distance

            # Zapis historii (zawsze zapisujemy stan po iteracji)
            history_cost.append(current_distance)
            history_best_cost.append(best_distance)
            iteration_count += 1

        # Chłodzenie (logika bez zmian)
        epoch_count += 1
        if cooling_type == 'exp':
            temp *= cooling_param
        elif cooling_type == 'lin':
             temp -= cooling_param
        elif cooling_type == 'log':
            log_arg = 1 + epoch_count
            if log_arg > 0 and cooling_param > 0:
                # Ensure temp decreases, handle potential precision issues near T=0 for log
                new_temp = start_temp_for_log / (1 + cooling_param * math.log(log_arg))
                temp = new_temp if new_temp < temp else temp * 0.99 # Force decrease if stagnant due to precision
            else:
                print("Warning: Log cooling encountered issue.")
                temp = 0
        else:
             raise ValueError(f"Unknown cooling type: {cooling_type}")

        if temp < 0: temp = 0

        # Wydruk postępu (bez zmian)
        if iteration_count % (iterations_per_temp * 5000) == 0 or epoch_count == 1:
             print(f"  Iter: {iteration_count}, Epoch: {epoch_count}, Temp: {temp:.4f}, Current Dist: {current_distance:.2f}, Best Dist: {best_distance:.2f}")

    print(f"Finished SA. Stop reason: temp ({temp:.4f}) <= effective_final_temp ({effective_final_temp:.4f}).")
    print(f"Iterations: {iteration_count}, Epochs: {epoch_count}, Final Temp: {temp:.4f}, Best distance found: {best_distance:.2f}")
    return best_order, best_distance, history_cost, history_best_cost, history_temp


# --- Funkcje do wizualizacji ---
# (bez zmian - plot_points, plot_solution, plot_convergence)
def plot_points(points, title, filename):
    plt.figure(figsize=(8, 8))
    plt.scatter(points[:, 0], points[:, 1], c='blue', marker='o', s=20) # Mniejsze punkty
    plt.title(title)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.grid(True)
    plt.axis('equal')
    plt.savefig(filename)
    plt.close()

def plot_solution(points, order, title, filename):
    plt.figure(figsize=(10, 10))
    n_points = len(points)
    if n_points == 0 or order is None or len(order) == 0:
        #print(f"Warning: Cannot plot solution for {filename}, no points or order.")
        plt.close()
        return
    try:
        # Sprawdzenie czy wszystkie indeksy w 'order' są poprawne
        if not all(0 <= i < n_points for i in order):
             print(f"Error: Invalid indices found in 'order' for {filename}.")
             plt.close()
             return
        if len(order) != n_points:
             print(f"Error: 'order' length ({len(order)}) does not match number of points ({n_points}) for {filename}.")
             # Try to fix if it's just missing points? No, better to fail.
             plt.close()
             return
        ordered_points = points[order]
    except IndexError as e:
        print(f"Error creating ordered_points: {e} for {filename}. Order: {order}, Points shape: {points.shape}")
        plt.close()
        return
    except TypeError as e:
         print(f"Error (TypeError) creating ordered_points: {e} for {filename}. Order: {order}, Points shape: {points.shape}")
         plt.close()
         return


    plt.scatter(points[:, 0], points[:, 1], c='lightblue', marker='o', s=50, label='Miasta', zorder=2)
    if ordered_points.size > 0:
         plt.scatter(ordered_points[0, 0], ordered_points[0, 1], c='lime', marker='s', s=100, label='Start/Koniec', zorder=3)

    for i in range(n_points):
        start_node = ordered_points[i]
        end_node = ordered_points[(i + 1) % n_points]

        plt.plot([start_node[0], end_node[0]], [start_node[1], end_node[1]],
                 c='red', linestyle='-', linewidth=1.0, zorder=1)

        dx = end_node[0] - start_node[0]
        dy = end_node[1] - start_node[1]

        segment_length = math.sqrt(dx**2 + dy**2)
        if segment_length > 1e-2:
            arrow_scale = min(0.2 * segment_length, 5.0)
            head_width = max(arrow_scale * 0.4, 0.1)
            head_length = max(arrow_scale * 0.6, 0.15)

            arrow_start_x = start_node[0] + dx * 0.5
            arrow_start_y = start_node[1] + dy * 0.5
            arrow_dx = dx * 0.5
            arrow_dy = dy * 0.5

            # Check if arrow vector is non-zero before drawing
            if abs(arrow_dx) > 1e-6 or abs(arrow_dy) > 1e-6:
                 try:
                     plt.arrow(arrow_start_x, arrow_start_y, arrow_dx, arrow_dy,
                          head_width=head_width, head_length=head_length,
                          fc='red', ec='red', length_includes_head=True,
                          shape='full', overhang=0.1,
                          zorder=1)
                 except Exception as arrow_err:
                      print(f"Warning: Could not draw arrow for segment {i} in {filename}. Error: {arrow_err}")


    plt.title(title, fontsize=12)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()


def plot_convergence(costs, best_costs, title, filename):
    plt.figure(figsize=(12, 6))
    iterations = range(len(costs))
    plt.plot(iterations, costs, label='Bieżący koszt trasy', alpha=0.6, linewidth=0.8)
    plt.plot(iterations, best_costs, label='Najlepszy znaleziony koszt', color='red', linewidth=1.5)
    plt.title(title, fontsize=12)
    plt.xlabel("Iteracja")
    plt.ylabel("Całkowity dystans (Koszt)")
    plt.legend()
    plt.grid(True)
    valid_best_costs = [c for c in best_costs if c is not None and c > 0]
    if valid_best_costs and max(valid_best_costs) > 10 * min(valid_best_costs):
         try:
             plt.yscale('log')
         except ValueError as e:
              print(f"Warning: Could not set log scale for convergence plot {filename}. Error: {e}")
              # Attempt linear scale instead
              plt.yscale('linear')
    else:
         plt.yscale('linear') # Explicitly set linear if log is not suitable

    plt.tight_layout()
    plt.savefig(filename)
    plt.close()


# --- Główna pętla eksperymentu ---
point_generators = {
    "uniform": (generate_uniform_points, {}),
    "normal_clusters": (generate_normal_clusters, {'num_clusters': 4}),
    "separated_groups": (generate_separated_groups, {'num_groups': 9})
}

neighbor_methods = {
    "consecutive_swap": generate_neighbor_consecutive_swap,
    "arbitrary_swap": generate_neighbor_arbitrary_swap
}

cooling_schedules = {}
for rate in COOLING_RATES_EXP:
    cooling_schedules[f"exponential_{rate:.2f}"] = ('exp', rate)
cooling_schedules["linear"] = ('lin', COOLING_RATE_LIN)
cooling_schedules["logarithmic"] = ('log', COOLING_RATE_LOG)


# Główna pętla z dodatkową pętlą po funkcjach akceptacji
for n in N_VALUES:
    n_dir = os.path.join(OUTPUT_DIR, f"n_{n}")
    os.makedirs(n_dir, exist_ok=True)
    print(f"\n----- Rozpoczynanie testów dla n = {n} -----")

    for dist_name, (generator_func, gen_params) in point_generators.items():
        dist_dir = os.path.join(n_dir, dist_name)
        os.makedirs(dist_dir, exist_ok=True)
        print(f"\n  --- Rozkład punktów: {dist_name} ---")

        try:
            points = generator_func(n, **gen_params)
            # Lepsze sprawdzanie poprawności wygenerowanych punktów
            if not isinstance(points, np.ndarray) or points.ndim != 2 or points.shape[1] != 2:
                 print(f"Error: Point generator for n={n}, dist={dist_name} returned invalid shape or type. Skipping.")
                 continue
            if n > 0 and points.shape[0] != n:
                 print(f"Warning: Generated {points.shape[0]} points, expected {n} for {dist_name}. Adjusting.")
                 if points.shape[0] < n:
                     missing = n - points.shape[0]
                     # Generate extra points more carefully
                     extra_points = generate_uniform_points(missing, *gen_params.get('x_range',(0,100)), *gen_params.get('y_range',(0,100)))
                     if extra_points.size > 0:
                          points = np.vstack([points, extra_points]) if points.size > 0 else extra_points
                     else: # Failed to generate extra points
                          print("Could not generate extra points. Skipping combination.")
                          continue
                 else: # Za dużo
                     points = points[:n, :]
            elif n == 0 and points.shape[0] != 0:
                 points = np.array([]).reshape(0,2) # Ensure correct empty shape
            if points.shape[0] != n:
                 print(f"Error: Final point count ({points.shape[0]}) differs from target ({n}). Skipping.")
                 continue

        except Exception as e:
            print(f"Error generating points for n={n}, dist={dist_name}: {e}")
            traceback.print_exc()
            continue

        points_title = f"Układ punktów (n={n}, {dist_name})"
        points_filename = os.path.join(dist_dir, f"points_{dist_name}.png")
        if n > 0:
             plot_points(points, points_title, points_filename)
        else:
             print("Skipping plotting points for n=0.")


        for neighbor_name, neighbor_func in neighbor_methods.items():
            neighbor_dir = os.path.join(dist_dir, neighbor_name)
            os.makedirs(neighbor_dir, exist_ok=True)
            print(f"\n    -- Metoda sąsiada: {neighbor_name} --")

            for cooling_name, (cooling_type, cooling_param) in cooling_schedules.items():
                cooling_dir = os.path.join(neighbor_dir, cooling_name)
                os.makedirs(cooling_dir, exist_ok=True)
                print(f"\n      - Schemat chłodzenia: {cooling_name} -")

                # <<< NOWA PĘTLA: Iteracja po funkcjach akceptacji >>>
                for acceptance_func_name in ACCEPTANCE_FUNCTIONS:
                    acceptance_dir = os.path.join(cooling_dir, acceptance_func_name)
                    os.makedirs(acceptance_dir, exist_ok=True)
                    print(f"\n        * Funkcja akceptacji: {acceptance_func_name} *")

                    if n == 0:
                        print("        Skipping SA for n=0.")
                        continue

                    start_time = time.time()

                    try:
                        # Przekazujemy nową nazwę funkcji akceptacji
                        best_order, best_dist, history_cost, history_best_cost, history_temp = simulated_annealing(
                            points,
                            initial_temp=INITIAL_TEMP,
                            final_temp_default=FINAL_TEMP,
                            final_temp_log=FINAL_TEMP_LOG,
                            cooling_type=cooling_type,
                            cooling_param=cooling_param,
                            acceptance_func=acceptance_func_name, # <<< Przekazanie nazwy funkcji
                            iterations_per_temp=ITERATIONS_PER_TEMP,
                            neighbor_func=neighbor_func
                        )
                    except Exception as e:
                         print(f"!!!!!! Error during SA execution (n={n}, dist={dist_name}, neighbor={neighbor_name}, cool={cooling_name}, accept={acceptance_func_name}) !!!!!: {e}")
                         traceback.print_exc()
                         # Przypisz puste wartości, aby uniknąć błędów w plotowaniu
                         best_order, best_dist, history_cost, history_best_cost, history_temp = [], float('inf'), [], [], []
                         # continue # Można pominąć resztę dla tej kombinacji

                    end_time = time.time()
                    exec_time = end_time - start_time
                    print(f"        Czas wykonania: {exec_time:.2f} s")
                    print(f"        Najlepszy dystans: {best_dist:.2f}")

                    # --- Generowanie i zapisywanie wykresów (w folderze funkcji akceptacji) ---

                    # 1. Wizualizacja rozwiązania
                    # Dodano acceptance_func_name do tytułu i nazwy pliku
                    solution_title = f"TSP Solution (n={n}, {dist_name})\nNeighbor: {neighbor_name}, Cool: {cooling_name}, Accept: {acceptance_func_name}\nDist: {best_dist:.2f}, Time: {exec_time:.1f}s"
                    solution_filename = os.path.join(acceptance_dir, f"solution_{n}_{dist_name}_{neighbor_name}_{cooling_name}_{acceptance_func_name}.png")
                    try:
                         # Sprawdź, czy mamy poprawne dane przed rysowaniem
                         if best_order is not None and len(best_order) == n and n > 0:
                              plot_solution(points, best_order, solution_title, solution_filename)
                         elif n==0:
                              pass # Nic do narysowania dla n=0
                         else:
                              print(f"Skipping solution plot for {solution_filename} due to invalid 'best_order'.")
                    except Exception as e:
                        print(f"Error plotting solution for {solution_filename}: {e}")
                        traceback.print_exc()


                    # 2. Wizualizacja zbieżności
                    convergence_title = f"SA Convergence (n={n}, {dist_name})\nNeighbor: {neighbor_name}, Cool: {cooling_name}, Accept: {acceptance_func_name}"
                    convergence_filename = os.path.join(acceptance_dir, f"convergence_{n}_{dist_name}_{neighbor_name}_{cooling_name}_{acceptance_func_name}.png")
                    try:
                         if history_cost and history_best_cost:
                             plot_convergence(history_cost, history_best_cost, convergence_title, convergence_filename)
                         else:
                             print(f"Skipping convergence plot for {convergence_filename}: No history data.")
                    except Exception as e:
                        print(f"Error plotting convergence for {convergence_filename}: {e}")
                        traceback.print_exc()

                    # 3. Wykres temperatury
                    temp_title = f"Cooling Schedule (n={n}, {dist_name})\nNeighbor: {neighbor_name}, Cool: {cooling_name}, Accept: {acceptance_func_name}"
                    temp_filename = os.path.join(acceptance_dir, f"temperature_{n}_{dist_name}_{neighbor_name}_{cooling_name}_{acceptance_func_name}.png")
                    try:
                        if history_temp:
                            plt.figure(figsize=(10, 5))
                            plt.plot(range(len(history_temp)), history_temp)
                            plt.axhline(y=FINAL_TEMP, color='gray', linestyle='--', linewidth=0.8, label=f'T_final_default ({FINAL_TEMP:.1e})')
                            if cooling_type == 'log':
                                 plt.axhline(y=FINAL_TEMP_LOG, color='purple', linestyle=':', linewidth=1.0, label=f'T_final_log ({FINAL_TEMP_LOG:.1f})')

                            plt.title(temp_title)
                            plt.xlabel("Epoka (krok chłodzenia)")
                            plt.ylabel("Temperatura")
                            plt.legend()
                            plt.grid(True)
                            valid_temps = [t for t in history_temp if t is not None and t > 0]
                            if valid_temps and max(valid_temps) > 10 * min(valid_temps):
                                 try:
                                     plt.yscale('log')
                                 except ValueError as e:
                                     print(f"Warning: Could not set log scale for temperature plot {temp_filename}. Error: {e}")
                                     plt.yscale('linear')
                            else:
                                 plt.yscale('linear')
                            plt.tight_layout()
                            plt.savefig(temp_filename)
                            plt.close()
                        else:
                             print(f"Skipping temperature plot for {temp_filename}: No history data.")
                    except Exception as e:
                        print(f"Error plotting temperature for {temp_filename}: {e}")
                        traceback.print_exc()


print("\n----- Zakończono wszystkie eksperymenty -----")
print(f"Wyniki zostały zapisane w folderze: {OUTPUT_DIR}")


----- Rozpoczynanie testów dla n = 10 -----

  --- Rozkład punktów: uniform ---

    -- Metoda sąsiada: consecutive_swap --

      - Schemat chłodzenia: exponential_1.00 -

        * Funkcja akceptacji: metropolis *
Starting SA: n=10, T_init=10000.0, T_final=1.0e-03, Cooling=exp(0.999), Acceptance=metropolis, Neighbor=generate_neighbor_consecutive_swap, Iter/Temp=1
  Iter: 1, Epoch: 1, Temp: 9990.0000, Current Dist: 613.14, Best Dist: 613.14
  Iter: 5000, Epoch: 5000, Temp: 67.2111, Current Dist: 554.52, Best Dist: 347.82
  Iter: 10000, Epoch: 10000, Temp: 0.4517, Current Dist: 324.88, Best Dist: 324.88
  Iter: 15000, Epoch: 15000, Temp: 0.0030, Current Dist: 324.88, Best Dist: 324.88
Finished SA. Stop reason: temp (0.0010) <= effective_final_temp (0.0010).
Iterations: 16111, Epochs: 16111, Final Temp: 0.0010, Best distance found: 324.88
        Czas wykonania: 0.30 s
        Najlepszy dystans: 324.88

        * Funkcja akceptacji: logistic *
Starting SA: n=10, T_init=10000.0, T_final

In [5]:
# --- Konfiguracja ---
OUTPUT_DIR_IMG = "sa_image_results_swap_loop_256" # Nowy folder
os.makedirs(OUTPUT_DIR_IMG, exist_ok=True)

# Parametry obrazu
N_VALUES = [256] # Utrzymujemy mniejsze n dla testów
DELTAS = [0.1, 0.3, 0.4]

# Parametry SA dla obrazów
INITIAL_TEMP_IMG = 6 # Może wymagać dostrojenia dla swap
FINAL_TEMP_IMG = 0.1
COOLING_RATE_IMG = 0.99 # Można lekko przyspieszyć chłodzenie
# Iteracje mogą wymagać dostrojenia dla swap, zacznijmy od 1
ITERATIONS_FACTOR = 1

# Parametry GIF
CREATE_GIFS = True
GIF_FRAME_INTERVAL = 10
# <<< NOWOŚĆ: Parametr pętli GIF >>>
GIF_LOOP = 0 # 0 oznacza nieskończoną pętlę

# --- Definicje Sąsiedztwa ---
NEIGHBORHOOD_4 = 'von_neumann'
NEIGHBORHOOD_8 = 'moore'
NEIGHBORHOOD_RING2 = 'ring_2'
NEIGHBORHOODS = [NEIGHBORHOOD_4, NEIGHBORHOOD_8, NEIGHBORHOOD_RING2]

# --- Definicje Funkcji Energii ---
ENERGY_ISING = 'ising'
ENERGY_CONTOUR = 'contour' # Uwaga: Contour może być mniej sensowny ze swapem
ENERGY_CHECKERBOARD = 'checkerboard'
ENERGY_ATTRACT_REPEL = 'attract_repel'
ENERGY_FUNCTIONS = [ENERGY_ISING, ENERGY_CHECKERBOARD, ENERGY_ATTRACT_REPEL] # Usunięto Contour tymczasowo

# --- Funkcje Pomocnicze ---

def create_initial_state(n, delta):
    """Tworzy losowy obraz binarny n x n z gęstością delta."""
    if n <= 0: return np.array([]).reshape(0,2)
    image = np.zeros((n, n), dtype=int)
    num_black_pixels = int(round(n * n * delta))
    if num_black_pixels > n*n: num_black_pixels = n*n
    if num_black_pixels < 0: num_black_pixels = 0

    if num_black_pixels > 0 and num_black_pixels < n*n :
        indices = np.random.choice(n * n, num_black_pixels, replace=False)
        rows, cols = np.unravel_index(indices, (n, n))
        image[rows, cols] = 1
    elif num_black_pixels == n*n:
        image.fill(1)

    # Sprawdźmy faktyczną gęstość (ważne przy zaokrągleniach)
    actual_black = np.sum(image)
    # print(f"Initial state created n={n}, requested_delta={delta}, num_black={num_black_pixels}, actual_black={actual_black}")
    return image

def get_neighbors(r, c, n, neighborhood_type):
    """Zwraca listę współrzędnych sąsiadów (r, c) z periodycznymi warunkami."""
    neighbors = []
    coords = set() # Użyj setu, aby uniknąć duplikatów

    if neighborhood_type == NEIGHBORHOOD_4:
        potential_neighbors = [(r-1, c), (r+1, c), (r, c-1), (r, c+1)]
    elif neighborhood_type == NEIGHBORHOOD_8:
        potential_neighbors = [
            (r-1, c), (r+1, c), (r, c-1), (r, c+1),
            (r-1, c-1), (r-1, c+1), (r+1, c-1), (r+1, c+1)
        ]
    elif neighborhood_type == NEIGHBORHOOD_RING2:
        potential_neighbors = []
        for dr in range(-2, 3):
            for dc in range(-2, 3):
                if max(abs(dr), abs(dc)) == 2:
                     potential_neighbors.append((r + dr, c + dc))
    else:
        raise ValueError(f"Nieznany typ sąsiedztwa: {neighborhood_type}")

    for nr, nc in potential_neighbors:
        # Zastosuj periodyczne warunki i dodaj do setu
        coords.add((nr % n, nc % n))

    # Usuń centralny piksel, jeśli przez przypadek się tam znalazł (nie powinien przy tych definicjach)
    coords.discard((r % n, c % n))

    return list(coords)

# --- Nowy Generator Sąsiadów (Swap) ---
def generate_neighbor_swap(image, n, max_attempts=100):
    """
    Wybiera dwa losowe piksele o RÓŻNYCH kolorach.
    Zwraca ((r1, c1), (r2, c2)) lub None, jeśli nie znaleziono po max_attempts.
    """
    if n == 0: return None
    attempts = 0
    while attempts < max_attempts:
        r1, c1 = random.randint(0, n-1), random.randint(0, n-1)
        r2, c2 = random.randint(0, n-1), random.randint(0, n-1)

        # Upewnij się, że to różne piksele
        if r1 == r2 and c1 == c2:
            attempts += 1
            continue

        val1 = image[r1, c1]
        val2 = image[r2, c2]

        if val1 != val2:
            return ((r1, c1), (r2, c2)) # Znaleziono parę

        attempts += 1

    # print(f"Warning: Could not find pixels of different colors after {max_attempts} attempts.")
    return None # Nie udało się znaleźć pary

# --- Obliczanie Energii ---

# Funkcja energy_contribution pozostaje bez zmian
def energy_contribution(val1, val2, energy_func, is_far=False):
    """Wkład do energii pary pikseli (val1, val2)."""
    if energy_func == ENERGY_ISING:
        return -1.0 if val1 == val2 else 1.0
    elif energy_func == ENERGY_CHECKERBOARD:
        return 1.0 if val1 == val2 else -1.0
    elif energy_func == ENERGY_ATTRACT_REPEL:
        if not is_far: # Bliscy (jak Ising)
            return -1.0 if val1 == val2 else 1.0
        else: # Dalecy (jak Checkerboard)
            return 1.0 if val1 == val2 else -1.0
    # elif energy_func == ENERGY_CONTOUR: return 0.0 # Contour liczony inaczej
    else:
        return 0.0

# Funkcja calculate_total_energy musi być dostosowana do energii par
def calculate_total_energy(image, n, neighborhood_type, energy_func):
    """Oblicza CAŁKOWITĄ energię obrazu dla funkcji opartych na parach."""
    if n == 0: return 0.0
    total_energy = 0.0
    far_neighbors_cache = {} # Cache dla dalekich sąsiadów

    for r in range(n):
        for c in range(n):
            value = image[r, c]
            neighbors = get_neighbors(r, c, n, neighborhood_type)
            far_neighbors_list = []

            if energy_func == ENERGY_ATTRACT_REPEL:
                 # Optymalizacja: pobierz lub oblicz dalekich sąsiadów raz
                 if (r, c) not in far_neighbors_cache:
                     far_neighbors_cache[(r, c)] = get_neighbors(r, c, n, NEIGHBORHOOD_RING2)
                 far_neighbors_list = far_neighbors_cache[(r, c)]

            for nr, nc in neighbors:
                # Liczymy wkład pary (r,c) - (nr,nc)
                neighbor_value = image[nr, nc]
                is_far_flag = energy_func == ENERGY_ATTRACT_REPEL and ((nr,nc) in far_neighbors_list)
                total_energy += energy_contribution(value, neighbor_value, energy_func, is_far=is_far_flag)

    # Podziel przez 2, bo każdą parę policzyliśmy dwukrotnie
    return total_energy / 2.0


# --- Nowe Obliczanie Delta Energii (Swap) ---
def calculate_delta_energy_swap(image, p1, p2, n, neighborhood_type, energy_func):
    """Oblicza zmianę energii przy zamianie pikseli p1=(r1, c1) i p2=(r2, c2)."""
    r1, c1 = p1
    r2, c2 = p2
    val1_old = image[r1, c1]
    val2_old = image[r2, c2]

    # Wartości PO zamianie
    val1_new = val2_old
    val2_new = val1_old

    delta_e = 0.0

    # 1. Zmiana interakcji p1 z jego sąsiadami (bez p2)
    neighbors1 = get_neighbors(r1, c1, n, neighborhood_type)
    # Potrzebujemy też dalekich sąsiadów dla AttractRepel
    far_neighbors1 = get_neighbors(r1, c1, n, NEIGHBORHOOD_RING2) if energy_func == ENERGY_ATTRACT_REPEL else []

    for nr, nc in neighbors1:
        if (nr, nc) == (r2, c2): continue # Pomijamy bezpośrednią interakcję z p2 na razie
        neighbor_value = image[nr, nc]
        is_far = energy_func == ENERGY_ATTRACT_REPEL and (nr, nc) in far_neighbors1
        e_old = energy_contribution(val1_old, neighbor_value, energy_func, is_far)
        e_new = energy_contribution(val1_new, neighbor_value, energy_func, is_far)
        delta_e += (e_new - e_old)

    # 2. Zmiana interakcji p2 z jego sąsiadami (bez p1)
    neighbors2 = get_neighbors(r2, c2, n, neighborhood_type)
    far_neighbors2 = get_neighbors(r2, c2, n, NEIGHBORHOOD_RING2) if energy_func == ENERGY_ATTRACT_REPEL else []

    for nr, nc in neighbors2:
        if (nr, nc) == (r1, c1): continue # Pomijamy bezpośrednią interakcję z p1 na razie
        neighbor_value = image[nr, nc]
        is_far = energy_func == ENERGY_ATTRACT_REPEL and (nr, nc) in far_neighbors2
        e_old = energy_contribution(val2_old, neighbor_value, energy_func, is_far)
        e_new = energy_contribution(val2_new, neighbor_value, energy_func, is_far)
        delta_e += (e_new - e_old)

    # 3. Zmiana bezpośredniej interakcji między p1 i p2 (jeśli są sąsiadami)
    p2_is_neighbor_of_p1 = (r2, c2) in neighbors1 # Sprawdź, czy p2 jest sąsiadem p1

    if p2_is_neighbor_of_p1:
        # Czy p2 jest dalekim sąsiadem p1 dla AttractRepel?
        p2_is_far_neighbor = energy_func == ENERGY_ATTRACT_REPEL and (r2,c2) in far_neighbors1
        # Oblicz starą i nową interakcję
        e_old_direct = energy_contribution(val1_old, val2_old, energy_func, is_far=p2_is_far_neighbor)
        e_new_direct = energy_contribution(val1_new, val2_new, energy_func, is_far=p2_is_far_neighbor)
        # Dodaj zmianę (punkty 1 i 2 pominęły tę interakcję)
        delta_e += (e_new_direct - e_old_direct)

    return delta_e

# --- Symulowane Wyżarzanie (zmodyfikowane dla Swap) ---
def simulated_annealing_image_swap(initial_state, n, neighborhood_type, energy_func,
                              initial_temp, final_temp, cooling_rate, iterations_factor,
                              gif_writer=None, gif_interval=20):
    """Wykonuje algorytm SA dla obrazu binarnego używając SWAP."""
    current_state = initial_state.copy()
    # Oblicz energię początkową poprawnie
    current_energy = calculate_total_energy(current_state, n, neighborhood_type, energy_func)
    best_state = current_state.copy()
    best_energy = current_energy
    initial_black_count = np.sum(current_state) # Do weryfikacji

    temp = initial_temp
    epoch = 0
    iterations_per_temp = n * n * iterations_factor

    history_energy = [current_energy]
    history_best_energy = [best_energy]
    history_temp = [temp]

    print(f"Starting SA (Swap): n={n}, E_init={current_energy:.2f}, T_init={initial_temp:.2f}, N_type={neighborhood_type}, E_func={energy_func}, Initial Black={initial_black_count}")

    estimated_epochs = 0
    if cooling_rate < 1.0 and initial_temp > 0 and final_temp > 0:
         try:
             estimated_epochs = int(math.log(final_temp / initial_temp) / math.log(cooling_rate))
         except ValueError: estimated_epochs = 1000 # Fallback if T=0 or invalid log
    pbar = tqdm(total=estimated_epochs, desc=f"SA Progress (Swap, n={n})", unit="epoch", leave=False)


    while temp > final_temp and temp > 1e-6:
        accepted_moves = 0
        start_epoch_time = time.time()

        for _ in range(iterations_per_temp):
            # 1. Wygeneruj parę do zamiany
            swap_pair = generate_neighbor_swap(current_state, n)
            if swap_pair is None: # Nie znaleziono pary (może się zdarzyć przy małym n lub jednolitym obrazie)
                continue
            p1, p2 = swap_pair

            # 2. Oblicz zmianę energii (delta E) dla zamiany
            delta_energy = calculate_delta_energy_swap(current_state, p1, p2, n, neighborhood_type, energy_func)

            # 3. Podejmij decyzję o akceptacji (Metropolis)
            accept = False
            if delta_energy < 0:
                accept = True
            else:
                probability = math.exp(-delta_energy / temp) if temp > 1e-6 else 0.0
                if random.random() < probability:
                    accept = True

            # 4. Jeśli akceptowano, zaktualizuj stan i energię
            if accept:
                accepted_moves += 1
                # Zastosuj ZAMIANĘ
                r1, c1 = p1
                r2, c2 = p2
                current_state[r1, c1], current_state[r2, c2] = current_state[r2, c2], current_state[r1, c1]
                current_energy += delta_energy # Zaktualizuj energię o deltę

                # Weryfikacja (opcjonalna, spowalnia):
                # new_black_count = np.sum(current_state)
                # if new_black_count != initial_black_count:
                #    print(f"ERROR! Black count changed: {initial_black_count} -> {new_black_count}")
                #    # Można rzucić wyjątek lub zatrzymać

                # Zaktualizuj najlepsze rozwiązanie
                if current_energy < best_energy:
                    best_energy = current_energy
                    best_state = current_state.copy()

        # Koniec epoki
        epoch_time = time.time() - start_epoch_time
        acceptance_rate = accepted_moves / iterations_per_temp if iterations_per_temp > 0 else 0

        history_energy.append(current_energy)
        history_best_energy.append(best_energy)
        history_temp.append(temp)

        pbar.update(1)
        pbar.set_postfix({"Temp": f"{temp:.3f}", "Energy": f"{current_energy:.2f}", "Best E": f"{best_energy:.2f}", "Accept%": f"{acceptance_rate*100:.1f}"})

        if gif_writer is not None and epoch % gif_interval == 0:
             frame = np.kron(current_state, np.ones((4,4), dtype=int))
             gif_writer.append_data((frame * 255).astype(np.uint8))

        temp *= cooling_rate
        epoch += 1

    pbar.close()
    final_black_count = np.sum(best_state)
    print(f"Finished SA (Swap): Epochs={epoch}, Final T={temp:.4f}, Final E={current_energy:.2f}, Best E={best_energy:.2f}, Final Black={final_black_count} (Initial was {initial_black_count})")
    if initial_black_count != final_black_count:
         print("!!!!!!!!!!!!!!!! WARNING: Black pixel count changed !!!!!!!!!!!!!!!!")


    if gif_writer is not None:
         final_frame = np.kron(best_state, np.ones((4,4), dtype=int))
         gif_writer.append_data((final_frame * 255).astype(np.uint8))

    return best_state, best_energy, history_energy, history_best_energy, history_temp


# --- Funkcje Wizualizacji ---
# (plot_image, plot_energy_convergence - bez zmian)
def plot_image(image, title, filename):
    plt.figure(figsize=(8, 8))
    # Użyjemy 'binary' dla lepszego kontrastu 0/1 -> biały/czarny
    plt.imshow(image, cmap='binary', interpolation='none')
    plt.title(title, fontsize=10)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

def plot_energy_convergence(costs, best_costs, temps, title, filename):
    """Rysuje wykres zbieżności energii i temperatury."""
    fig, ax1 = plt.subplots(figsize=(12, 7))
    epochs = range(len(costs))

    color = 'tab:red'
    ax1.set_xlabel('Epoka')
    ax1.set_ylabel('Energia', color=color)
    # Rysuj od epoki 1 (pomijając stan początkowy dla lepszej skali Y)
    if len(epochs)>1:
        ax1.plot(epochs[1:], costs[1:], color=color, alpha=0.7, linewidth=0.9, label='Bieżąca Energia')
        ax1.plot(epochs[1:], best_costs[1:], color='black', linestyle='--', linewidth=1.2, label='Najlepsza Energia')
    else: # Handle case with only initial state
        ax1.plot(epochs, costs, color=color, alpha=0.7, linewidth=0.9, label='Bieżąca Energia')
        ax1.plot(epochs, best_costs, color='black', linestyle='--', linewidth=1.2, label='Najlepsza Energia')

    ax1.tick_params(axis='y', labelcolor=color)
    ax1.grid(True, axis='y', linestyle=':')
    ax1.legend(loc='upper left')

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Temperatura (log)', color=color)
    valid_temps_idx = [i for i, t in enumerate(temps) if t > 0]
    if valid_temps_idx:
       # Rysuj od epoki 1 również dla temperatury
       valid_epochs_plot = np.array(epochs)[valid_temps_idx]
       valid_temps_plot = np.array(temps)[valid_temps_idx]
       if len(valid_epochs_plot)>1 :
            ax2.plot(valid_epochs_plot[1:], valid_temps_plot[1:], color=color, alpha=0.6, label='Temperatura')
       elif len(valid_epochs_plot)>0: # Plot only first point if that's all
            ax2.plot(valid_epochs_plot, valid_temps_plot, color=color, alpha=0.6, label='Temperatura')

       try:
           ax2.set_yscale('log')
       except ValueError: # Handle cases where log scale is not possible
           ax2.set_yscale('linear')
       ax2.tick_params(axis='y', labelcolor=color)
       ax2.legend(loc='upper right')
    else:
         ax2.plot(epochs, temps, color=color, alpha=0.6, label='Temperatura')
         ax2.legend(loc='upper right')

    plt.title(title, fontsize=14)
    fig.tight_layout()
    plt.savefig(filename)
    plt.close()

# --- Główna Pętla Eksperymentu (zmodyfikowana do użycia SA Swap) ---

for n in N_VALUES:
    n_dir = os.path.join(OUTPUT_DIR_IMG, f"n_{n}")
    os.makedirs(n_dir, exist_ok=True)
    print(f"\n===== Processing N = {n} =====")

    for delta in DELTAS:
        delta_str = f"{delta:.1f}".replace('.', '_')
        delta_dir = os.path.join(n_dir, f"delta_{delta_str}")
        os.makedirs(delta_dir, exist_ok=True)
        print(f"\n  --- Processing Delta = {delta} ---")

        initial_state = create_initial_state(n, delta)
        if initial_state.size == 0 and n > 0:
            print("Error: Failed to create initial state. Skipping.")
            continue
        initial_img_filename = os.path.join(delta_dir, f"initial_n{n}_d{delta_str}.png")
        if n > 0 :
            plot_image(initial_state, f"Stan początkowy (n={n}, δ={delta})", initial_img_filename)

        for neighborhood in NEIGHBORHOODS:
            neigh_dir = os.path.join(delta_dir, neighborhood)
            os.makedirs(neigh_dir, exist_ok=True)
            print(f"\n    -- Processing Neighborhood = {neighborhood} --")

            for energy_func in ENERGY_FUNCTIONS:
                energy_dir = os.path.join(neigh_dir, energy_func)
                os.makedirs(energy_dir, exist_ok=True)
                print(f"\n      * Processing Energy Function = {energy_func} *")

                gif_filename = None
                gif_writer = None
                if CREATE_GIFS and n > 0:
                    gif_filename = os.path.join(energy_dir, f"evolution_n{n}_d{delta_str}_{neighborhood}_{energy_func}.gif")
                    try:
                        # <<< ZMIANA: Dodano kwargs={'loop': GIF_LOOP} >>>
                        gif_writer = imageio.get_writer(gif_filename, mode='I', duration=0.05, **{'loop': GIF_LOOP}) # Szybszy GIF, zapętlony
                        initial_frame = np.kron(initial_state, np.ones((4,4), dtype=int))
                        gif_writer.append_data((initial_frame * 255).astype(np.uint8))
                    except Exception as e:
                         print(f"Error creating GIF writer for {gif_filename}: {e}")
                         gif_writer = None
                elif n == 0:
                     print("Skipping GIF creation for n=0.")


                start_time = time.time()

                if n == 0: # Handle n=0 case explicitly
                    print("Skipping SA for n=0.")
                    best_state, best_energy = initial_state, 0.0
                    history_energy, history_best_energy, history_temp = [0.0], [0.0], []
                    exec_time = 0.0
                else:
                    try:
                        # <<< ZMIANA: Wywołujemy nową funkcję SA >>>
                        best_state, best_energy, history_energy, history_best_energy, history_temp = simulated_annealing_image_swap(
                            initial_state, n, neighborhood, energy_func,
                            INITIAL_TEMP_IMG, FINAL_TEMP_IMG, COOLING_RATE_IMG, ITERATIONS_FACTOR,
                            gif_writer, GIF_FRAME_INTERVAL
                        )
                    except Exception as e:
                        print(f"!!!!!! Error during SA Image Swap execution !!!!!: {e}")
                        traceback.print_exc()
                        best_state, best_energy = initial_state, float('inf')
                        history_energy, history_best_energy, history_temp = [], [], []

                    exec_time = time.time() - start_time

                print(f"      Execution Time: {exec_time:.2f} s")
                print(f"      Best Energy Found: {best_energy:.4f}")

                if gif_writer is not None:
                    try:
                        gif_writer.close()
                        print(f"      GIF saved to: {gif_filename}")
                    except Exception as e:
                         print(f"Error closing GIF writer for {gif_filename}: {e}")


                final_img_filename = os.path.join(energy_dir, f"final_n{n}_d{delta_str}_{neighborhood}_{energy_func}_E{best_energy:.2f}.png")
                if n > 0:
                    plot_image(best_state, f"Wynik SA Swap (n={n}, δ={delta}, N={neighborhood}, E={energy_func})\nBest Energy: {best_energy:.2f}", final_img_filename)

                conv_filename = os.path.join(energy_dir, f"convergence_n{n}_d{delta_str}_{neighborhood}_{energy_func}.png")
                if history_energy and history_best_energy and history_temp:
                     plot_energy_convergence(history_energy, history_best_energy, history_temp,
                                           f"Zbieżność Energii SA Swap (n={n}, δ={delta}, N={neighborhood}, E={energy_func})",
                                           conv_filename)
                else:
                     print("Skipping convergence plot - no history data.")


num_gifs_expected = len(N_VALUES) * len(DELTAS) * len(NEIGHBORHOODS) * len(ENERGY_FUNCTIONS)
print(f"\nExpected number of looping GIFs to be generated (if CREATE_GIFS is True): {num_gifs_expected}")

print("\n===== All experiments finished =====")
print(f"Results saved in: {OUTPUT_DIR_IMG}")


===== Processing N = 256 =====

  --- Processing Delta = 0.1 ---

    -- Processing Neighborhood = von_neumann --

      * Processing Energy Function = ising *
Starting SA (Swap): n=256, E_init=-83920.00, T_init=6.00, N_type=von_neumann, E_func=ising, Initial Black=6554


Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=-126848.00, Best E=-126848.00, Final Black=6554 (Initial was 6554)
      Execution Time: 667.64 s
      Best Energy Found: -126848.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_1\von_neumann\ising\evolution_n256_d0_1_von_neumann_ising.gif

      * Processing Energy Function = checkerboard *
Starting SA (Swap): n=256, E_init=83920.00, T_init=6.00, N_type=von_neumann, E_func=checkerboard, Initial Black=6554


Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=78640.00, Best E=78640.00, Final Black=6554 (Initial was 6554)
      Execution Time: 679.84 s
      Best Energy Found: 78640.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_1\von_neumann\checkerboard\evolution_n256_d0_1_von_neumann_checkerboard.gif

      * Processing Energy Function = attract_repel *
Starting SA (Swap): n=256, E_init=-83920.00, T_init=6.00, N_type=von_neumann, E_func=attract_repel, Initial Black=6554


Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=-126784.00, Best E=-126784.00, Final Black=6554 (Initial was 6554)
      Execution Time: 1167.29 s
      Best Energy Found: -126784.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_1\von_neumann\attract_repel\evolution_n256_d0_1_von_neumann_attract_repel.gif

    -- Processing Neighborhood = moore --

      * Processing Energy Function = ising *
Starting SA (Swap): n=256, E_init=-167732.00, T_init=6.00, N_type=moore, E_func=ising, Initial Black=6554


Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=-254408.00, Best E=-254408.00, Final Black=6554 (Initial was 6554)
      Execution Time: 739.05 s
      Best Energy Found: -254408.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_1\moore\ising\evolution_n256_d0_1_moore_ising.gif

      * Processing Energy Function = checkerboard *
Starting SA (Swap): n=256, E_init=167732.00, T_init=6.00, N_type=moore, E_func=checkerboard, Initial Black=6554


...

Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=-78484.00, Best E=-78484.00, Final Black=26214 (Initial was 26214)
      Execution Time: 493.56 s
      Best Energy Found: -78484.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_4\moore\checkerboard\evolution_n256_d0_4_moore_checkerboard.gif

      * Processing Energy Function = attract_repel *
Starting SA (Swap): n=256, E_init=-9652.00, T_init=6.00, N_type=moore, E_func=attract_repel, Initial Black=26214


Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=-244232.00, Best E=-244232.00, Final Black=26214 (Initial was 26214)
      Execution Time: 1147.91 s
      Best Energy Found: -244232.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_4\moore\attract_repel\evolution_n256_d0_4_moore_attract_repel.gif

    -- Processing Neighborhood = ring_2 --

      * Processing Energy Function = ising *
Starting SA (Swap): n=256, E_init=-19668.00, T_init=6.00, N_type=ring_2, E_func=ising, Initial Black=26214


Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=-479656.00, Best E=-479656.00, Final Black=26214 (Initial was 26214)
      Execution Time: 1235.77 s
      Best Energy Found: -479656.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_4\ring_2\ising\evolution_n256_d0_4_ring_2_ising.gif

      * Processing Energy Function = checkerboard *
Starting SA (Swap): n=256, E_init=19668.00, T_init=6.00, N_type=ring_2, E_func=checkerboard, Initial Black=26214


Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=-159320.00, Best E=-159320.00, Final Black=26214 (Initial was 26214)
      Execution Time: 1935.36 s
      Best Energy Found: -159320.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_4\ring_2\checkerboard\evolution_n256_d0_4_ring_2_checkerboard.gif

      * Processing Energy Function = attract_repel *
Starting SA (Swap): n=256, E_init=19668.00, T_init=6.00, N_type=ring_2, E_func=attract_repel, Initial Black=26214


Finished SA (Swap): Epochs=408, Final T=0.0994, Final E=-159236.00, Best E=-159236.00, Final Black=26214 (Initial was 26214)
      Execution Time: 4285.10 s
      Best Energy Found: -159236.0000
      GIF saved to: sa_image_results_swap_loop_256\n_256\delta_0_4\ring_2\attract_repel\evolution_n256_d0_4_ring_2_attract_repel.gif

Expected number of looping GIFs to be generated (if CREATE_GIFS is True): 27

===== All experiments finished =====
Results saved in: sa_image_results_swap_loop_256


In [13]:


# --- Konfiguracja ---
OUTPUT_DIR_SUDOKU = "sudoku_sa_analysis_robust_io" # Nowy folder
os.makedirs(OUTPUT_DIR_SUDOKU, exist_ok=True)

# --- Konfiguracja SA ---
INITIAL_TEMP = 1.0
FINAL_TEMP = 0.01
COOLING_RATE = 0.99
ITERATIONS_PER_TEMP = 100

# --- Funkcje Pomocnicze ---

def print_grid(grid, title="Sudoku Grid"):
    """Wyświetla planszę Sudoku."""
    if grid is None or grid.shape != (9,9):
         print(f"Invalid grid provided to print_grid for title: {title}")
         return
    print(f"--- {title} ---")
    for r in range(9):
        if r % 3 == 0 and r != 0: print("-" * 21)
        row_str = ""
        for c in range(9):
            if c % 3 == 0 and c != 0: row_str += "| "
            val = grid[r, c]
            row_str += (str(val) if val != 0 else '.') + " "
        print(row_str.strip())
    print("-" * 21)

# <<< ZMODYFIKOWANA FUNKCJA ODCZYTU >>>
def read_sudoku(filename):
    """Wczytuje planszę Sudoku z pliku, bardziej odporna na formatowanie."""
    try:
        with open(filename, 'r') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"Błąd: Plik '{filename}' nie znaleziony.")
        return None, None
    except Exception as e:
         print(f"Błąd podczas otwierania pliku {filename}: {e}")
         return None, None

    # Usuń komentarze (linie zaczynające się od #) i białe znaki (spacje, tabulatory, newline)
    # Zachowaj tylko cyfry 0-9 i kropki '.' (jako puste)
    # 'x' też można dopuścić i zamienić na '.'
    cleaned_content = re.sub(r'#.*', '', content) # Usuń komentarze
    cleaned_content = re.sub(r'\s+', '', cleaned_content) # Usuń wszystkie białe znaki
    cleaned_content = cleaned_content.replace('x', '.').replace('X', '.') # Zamień 'x' na '.'

    # Sprawdź, czy zostały tylko dozwolone znaki
    allowed_chars = set('0123456789.')
    processed_chars = ""
    for char in cleaned_content:
        if char in allowed_chars:
            processed_chars += char
        # else: # Można dodać ostrzeżenie o ignorowanych znakach
        #     print(f"Warning: Ignored unexpected character '{char}' in {filename}")


    if len(processed_chars) != 81:
        print(f"Błąd: Po przetworzeniu pliku '{filename}' uzyskano {len(processed_chars)} znaków zamiast 81. Zawartość po czyszczeniu: '{processed_chars[:100]}...'")
        return None, None

    # Utwórz planszę i maskę
    grid = np.zeros((9, 9), dtype=int)
    fixed_mask = np.zeros((9, 9), dtype=bool)

    try:
        for i in range(81):
            r, c = divmod(i, 9) # Oblicz rząd i kolumnę
            char = processed_chars[i]
            if char == '.':
                grid[r, c] = 0
                fixed_mask[r, c] = False
            elif char.isdigit():
                val = int(char)
                # Pozwól na 0 jako puste pole
                if val == 0:
                     grid[r, c] = 0
                     fixed_mask[r, c] = False
                elif 1 <= val <= 9:
                    grid[r, c] = val
                    fixed_mask[r, c] = True
                else:
                     # Ten warunek nie powinien być osiągnięty po filtrowaniu, ale dla bezpieczeństwa
                     print(f"Błąd: Nieprawidłowa cyfra '{char}' napotkana po przetworzeniu pliku '{filename}' (pozycja {i}).")
                     return None, None
            # Inne znaki nie powinny przejść filtrowania
    except Exception as e:
        print(f"Błąd podczas tworzenia planszy z przetworzonej zawartości pliku '{filename}': {e}")
        return None, None


    return grid, fixed_mask

# <<< ZMODYFIKOWANA FUNKCJA ZAPISU >>>
def write_sudoku_file(filename, grid_string_81):
    """Zapisuje string 81 znaków do pliku Sudoku (9x9)."""
    if len(grid_string_81) != 81:
        print(f"Błąd: Próba zapisu do {filename} niepoprawnej długości stringa ({len(grid_string_81)} zamiast 81).")
        return False

    # Zamień ewentualne 'x' na '.' dla spójności
    grid_string_81 = grid_string_81.replace('x', '.').replace('X', '.')

    try:
        with open(filename, "w") as f:
            for i in range(0, 81, 9):
                f.write(grid_string_81[i:i+9] + "\n")
        # print(f"Plik {filename} zapisany/nadpisany pomyślnie.")
        return True
    except Exception as e:
        print(f"Błąd zapisu do pliku {filename}: {e}")
        return False


# --- Funkcje fill_initial_blanks, calculate_cost, generate_neighbor, SA (bez zmian) ---
def fill_initial_blanks(grid, fixed_mask):
    filled_grid = grid.copy(); n=9; block_size=3
    for r_block_start in range(0,n,block_size):
        for c_block_start in range(0,n,block_size):
            block_fixed_nums=set(); block_blanks_coords=[]
            for r in range(r_block_start,r_block_start+block_size):
                for c in range(c_block_start,c_block_start+block_size):
                    if fixed_mask[r,c]: block_fixed_nums.add(filled_grid[r,c])
                    elif filled_grid[r,c]==0: block_blanks_coords.append((r,c))
            missing_nums=list(set(range(1,n+1))-block_fixed_nums); random.shuffle(missing_nums)
            if len(missing_nums)==len(block_blanks_coords):
                for i,(r,c) in enumerate(block_blanks_coords): filled_grid[r,c]=missing_nums[i]
            else:
                available_digits=list(range(1,n+1))
                for r,c in block_blanks_coords: filled_grid[r,c]=random.choice(available_digits)
    if np.any(filled_grid==0):
        for r in range(n):
            for c in range(n):
                 if filled_grid[r,c]==0 and not fixed_mask[r,c]: filled_grid[r,c]=random.randint(1,9)
    return filled_grid

def calculate_cost(grid):
    cost = 0; n = 9
    if grid is None or grid.shape != (9,9): return float('inf') # Handle invalid grid
    if np.any(grid < 0) or np.any(grid > 9): return float('inf') # Invalid numbers
    if np.any(grid == 0): cost += np.sum(grid == 0) * 10 # Penalize incomplete grids

    for r in range(n):
        row=grid[r,:]; non_zeros=row[row!=0]; counts=Counter(non_zeros)
        for digit in range(1,n+1):
            if counts[digit]>1: cost+=(counts[digit]-1)
    for c in range(n):
        col=grid[:,c]; non_zeros=col[col!=0]; counts=Counter(non_zeros)
        for digit in range(1,n+1):
            if counts[digit]>1: cost+=(counts[digit]-1)
    for r_block in range(0,n,3):
        for c_block in range(0,n,3):
            block=grid[r_block:r_block+3,c_block:c_block+3].flatten(); non_zeros=block[block!=0]; counts=Counter(non_zeros)
            for digit in range(1,n+1):
                if counts[digit]>1: cost+=(counts[digit]-1)
    return cost

def generate_neighbor(grid, fixed_mask):
    new_grid=grid.copy(); n=9
    modifiable_cells=[(r,c) for r in range(n) for c in range(n) if not fixed_mask[r,c]]
    if len(modifiable_cells)<2: return new_grid
    attempts=0; swapped=False
    while attempts<5 and not swapped:
        block_r_start=random.choice([0,3,6]); block_c_start=random.choice([0,3,6])
        block_modifiable_cells=[(r,c) for (r,c) in modifiable_cells if block_r_start<=r<block_r_start+3 and block_c_start<=c<block_c_start+3]
        if len(block_modifiable_cells)>=2:
            idx1,idx2=random.sample(range(len(block_modifiable_cells)),2); r1,c1=block_modifiable_cells[idx1]; r2,c2=block_modifiable_cells[idx2]
            new_grid[r1,c1],new_grid[r2,c2]=new_grid[r2,c2],new_grid[r1,c1]
            swapped=True; break
        attempts+=1
    if not swapped:
        idx1,idx2=random.sample(range(len(modifiable_cells)),2); r1,c1=modifiable_cells[idx1]; r2,c2=modifiable_cells[idx2]
        new_grid[r1,c1],new_grid[r2,c2]=new_grid[r2,c2],new_grid[r1,c1]
    return new_grid

def simulated_annealing_sudoku(initial_grid, fixed_mask, initial_temp, final_temp, cooling_rate, iterations_per_temp):
    current_grid=fill_initial_blanks(initial_grid,fixed_mask); current_cost=calculate_cost(current_grid)
    best_grid=current_grid.copy(); best_cost=current_cost
    if best_cost==0 and not np.any(initial_grid==0): return best_grid,best_cost,0,[0]
    temp=initial_temp; total_iterations=0; start_time=time.time(); cost_history=[current_cost]; solved=False
    print(f"Starting SA: Initial Cost={current_cost}, T_init={initial_temp:.2f}")
    while temp>final_temp:
        accepted_moves=0
        if best_cost==0: solved=True; break
        for i in range(iterations_per_temp):
            total_iterations+=1; neighbor_grid=generate_neighbor(current_grid,fixed_mask); neighbor_cost=calculate_cost(neighbor_grid)
            delta_cost=neighbor_cost-current_cost; accept=False
            if delta_cost<0: accept=True
            elif temp>1e-9:
                try:
                    probability=math.exp(-delta_cost/temp)
                    if random.random()<probability: accept=True
                except OverflowError: pass
            if accept:
                accepted_moves+=1; current_grid=neighbor_grid; current_cost=neighbor_cost
                if current_cost<best_cost:
                    best_cost=current_cost; best_grid=current_grid.copy()
                    if best_cost==0: print(f"\nSolution found at iteration {total_iterations}!"); solved=True; break
        cost_history.append(current_cost)
        if solved: break
        temp*=cooling_rate
        if total_iterations > 0 and (total_iterations // iterations_per_temp) % 20 == 0: print(f"  Iter: {total_iterations}, Temp: {temp:.4f}, Cost: {current_cost}, Best: {best_cost}")
    end_time=time.time(); elapsed_time=end_time-start_time; final_cost=calculate_cost(best_grid)
    print(f"Finished SA: Total Iter={total_iterations}, Time={elapsed_time:.2f}s, Final T={temp:.4f}, Best Cost={final_cost}")
    return best_grid,final_cost,total_iterations,cost_history

# --- Funkcje Wykresów (bez zmian) ---
def plot_cost_vs_blanks(results_data, output_dir):
    if not results_data: return
    results_data.sort(key=lambda x: x[0])
    blanks=[item[0] for item in results_data]; final_costs=[item[1] for item in results_data]; solved_status=['green' if cost==0 else 'red' for cost in final_costs]
    plt.figure(figsize=(12,7)); plt.scatter(blanks,final_costs,c=solved_status,s=60,alpha=0.7,edgecolors='k'); plt.axhline(0,color='grey',linestyle='--',linewidth=0.8)
    plt.xlabel("Liczba pustych miejsc na planszy"); plt.ylabel("Koszt końcowy (0 = rozwiązanie)"); plt.title("Zależność końcowego kosztu SA od trudności Sudoku")
    plt.grid(True,linestyle=':'); unique_blanks=sorted(list(set(blanks))); plt.xticks(unique_blanks if len(unique_blanks)<20 else np.linspace(min(blanks),max(blanks),10,dtype=int)); plt.ylim(bottom=-0.5)
    from matplotlib.lines import Line2D
    legend_elements=[Line2D([0],[0],marker='o',color='w',label='Rozwiązane (Koszt=0)',markerfacecolor='green',markersize=8,markeredgecolor='k'),Line2D([0],[0],marker='o',color='w',label='Nierozwiązane (Koszt>0)',markerfacecolor='red',markersize=8,markeredgecolor='k')]
    plt.legend(handles=legend_elements,loc='best'); plt.tight_layout(); plot_filename=os.path.join(output_dir,"sudoku_sa_final_cost_vs_blanks.png")
    try: plt.savefig(plot_filename); print(f"Wykres kosztu vs puste pola zapisano do pliku: {plot_filename}")
    except Exception as e: print(f"Error saving plot {plot_filename}: {e}")
    plt.close()

def plot_iterations_vs_blanks(results_data, output_dir):
    if not results_data: return
    results_data.sort(key=lambda x: x[0])
    blanks=[item[0] for item in results_data]; iterations=[item[2] for item in results_data]; final_costs=[item[1] for item in results_data]; solved_status=['green' if cost==0 else 'red' for cost in final_costs]
    plt.figure(figsize=(12,7)); plt.scatter(blanks,iterations,c=solved_status,s=60,alpha=0.7,edgecolors='k')
    plt.xlabel("Liczba pustych miejsc na planszy"); plt.ylabel("Całkowita liczba iteracji SA"); plt.title("Zależność liczby iteracji SA od trudności Sudoku")
    plt.grid(True,linestyle=':'); unique_blanks=sorted(list(set(blanks))); plt.xticks(unique_blanks if len(unique_blanks)<20 else np.linspace(min(blanks),max(blanks),10,dtype=int))
    # plt.yscale('log')
    from matplotlib.lines import Line2D
    legend_elements=[Line2D([0],[0],marker='o',color='w',label='Rozwiązane (Koszt=0)',markerfacecolor='green',markersize=8,markeredgecolor='k'),Line2D([0],[0],marker='o',color='w',label='Nierozwiązane (Koszt>0)',markerfacecolor='red',markersize=8,markeredgecolor='k')]
    plt.legend(handles=legend_elements,loc='best'); plt.tight_layout(); plot_filename=os.path.join(output_dir,"sudoku_sa_iterations_vs_blanks.png")
    try: plt.savefig(plot_filename); print(f"Wykres iteracji vs puste pola zapisano do pliku: {plot_filename}")
    except Exception as e: print(f"Error saving plot {plot_filename}: {e}")
    plt.close()

def plot_cost_history(cost_histories, num_blanks_list, output_dir):
    if not cost_histories: return
    plt.figure(figsize=(12,7)); max_len=0
    for history in cost_histories.values(): max_len=max(max_len,len(history))
    cmap=plt.get_cmap('viridis'); valid_blanks=list(cost_histories.keys()); norm=plt.Normalize(vmin=min(valid_blanks),vmax=max(valid_blanks))
    for num_blanks in sorted(cost_histories.keys()):
        history=cost_histories[num_blanks]
        if not history: continue
        epochs=range(len(history)); color=cmap(norm(num_blanks))
        plt.plot(epochs,history,label=f'Puste: {num_blanks}',color=color,alpha=0.8,linewidth=1)
    plt.xlabel("Epoka SA (przybliżona)"); plt.ylabel("Koszt (Liczba konfliktów)"); plt.title("Historia kosztu podczas działania SA dla różnych trudności Sudoku")
    plt.grid(True,linestyle=':'); plt.legend(title="Liczba pustych pól",bbox_to_anchor=(1.05,1),loc='upper left'); plt.yscale('log'); plt.ylim(bottom=0.5); plt.xlim(left=0,right=max_len if max_len>0 else 1)
    plt.tight_layout(rect=[0,0,0.85,1]); plot_filename=os.path.join(output_dir,"sudoku_sa_cost_history_comparison.png")
    try: plt.savefig(plot_filename); print(f"Wykres historii kosztów zapisano do pliku: {plot_filename}")
    except Exception as e: print(f"Error saving plot {plot_filename}: {e}")
    plt.close()

# --- Główna część programu ---

if __name__ == "__main__":
    # Definicje plansz Sudoku jako stringi 81 znaków ('.' dla pustych)
    sudoku_grids_81 = {
        "very_easy_27": ".6.1.4.5.1.9.8.67.8.6..24...5..7.7...6..1.3..4...2...6.6.28...7.9..4...5",
        "easy_36":      "53..7....6..195....98....6.8...6...34..8.3..17...2...6.6....28....419..5....8..79",
        "medium_45":    ".2.6.8...58...97......4....37...5.1.6...1...4.4.7...69....5......72...36...4.1.9.",
        "hard_50":      ".1...7.9..3...4.8...91...6...5.6.4....4.8.3.2...1.9.6....7...5.4..2.7...5..6.3...7.",
        "harder_54":    ".9.....4.7....1..8.2...5..6.8...75.3.7...6.1.5...2.4..3...9..4....5.2.....6.", 
        "evil_59":      "8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..",
        "evil_61":      "..53.....8......72....6.1..5.9.8.....46...7.1.....5.2........9....4..3....3......", 
        "top_95_1_51":  ".......3..2.3.5...1.4.6...7.........5.7.9.....3...6.1...........1.4.7..8.6.....",
        "blank_81":     "................................................................................."
    }

    print("Tworzenie/Nadpisywanie plików Sudoku...")
    valid_puzzles = {} # Słownik na poprawnie zapisane puzzle
    for name, grid_string in sudoku_grids_81.items():
        # Usuń potencjalne białe znaki z definicji stringa
        clean_grid_string = "".join(grid_string.split())
        if len(clean_grid_string) != 81:
             print(f"OSTRZEŻENIE: Definicja '{name}' ma niepoprawną długość ({len(clean_grid_string)}). Pomijanie.")
             continue

        filename = os.path.join(OUTPUT_DIR_SUDOKU, f"sudoku_{name}.txt")
        if write_sudoku_file(filename, clean_grid_string):
            valid_puzzles[name] = filename # Dodaj do listy tylko jeśli plik zapisano

    print("Pliki Sudoku gotowe.")
    if not valid_puzzles:
        print("Nie udało się utworzyć żadnych poprawnych plików Sudoku. Kończenie.")
        exit()

    results_data = []
    cost_histories = {}

    # Przetwarzanie plików
    for name, filename in valid_puzzles.items():
        print(f"\n--- Rozwiązywanie: {name} ({filename}) ---")
        initial_grid, fixed_mask = read_sudoku(filename)

        if initial_grid is None or fixed_mask is None:
            print("Pominięto z powodu błędu wczytywania.")
            continue

        num_blanks = np.sum(initial_grid == 0)
        print(f"Liczba pustych pól: {num_blanks}")

        try:
            if num_blanks > 0:
                solution_grid, final_cost, total_iterations, cost_history = simulated_annealing_sudoku(
                    initial_grid, fixed_mask,
                    INITIAL_TEMP, FINAL_TEMP, COOLING_RATE, ITERATIONS_PER_TEMP
                )
            else:
                 print("Plansza wejściowa jest pełna.")
                 solution_grid = initial_grid
                 final_cost = calculate_cost(solution_grid)
                 total_iterations = 0
                 cost_history = [final_cost]

            print("\nNajlepsze znalezione rozwiązanie:")
            print_grid(solution_grid, f"Best Found (Cost={final_cost})")

            if final_cost == 0: print("ZNALEZIONO POPRAWNE ROZWIĄZANIE!")
            else: print("Nie znaleziono poprawnego rozwiązania (koszt > 0).")

            results_data.append((num_blanks, final_cost, total_iterations))
            if num_blanks not in cost_histories: cost_histories[num_blanks] = cost_history

        except Exception as e:
            print(f"!!!!!! Wystąpił błąd podczas przetwarzania {name} !!!!!: {e}")
            traceback.print_exc()

    # --- Generowanie Wykresów ---
    print("\n--- Generowanie Wykresów ---")
    if results_data:
        plot_cost_vs_blanks(results_data, OUTPUT_DIR_SUDOKU)
        plot_iterations_vs_blanks(results_data, OUTPUT_DIR_SUDOKU)
        if cost_histories:
            num_blanks_list_for_plot = sorted(cost_histories.keys())
            plot_cost_history(cost_histories, num_blanks_list_for_plot, OUTPUT_DIR_SUDOKU)
    else:
        print("Brak wyników do wygenerowania wykresów.")

Tworzenie/Nadpisywanie plików Sudoku...
OSTRZEŻENIE: Definicja 'very_easy_27' ma niepoprawną długość (72). Pomijanie.
OSTRZEŻENIE: Definicja 'hard_50' ma niepoprawną długość (83). Pomijanie.
OSTRZEŻENIE: Definicja 'harder_54' ma niepoprawną długość (76). Pomijanie.
OSTRZEŻENIE: Definicja 'top_95_1_51' ma niepoprawną długość (79). Pomijanie.
Pliki Sudoku gotowe.

--- Rozwiązywanie: easy_36 (sudoku_sa_analysis_robust_io\sudoku_easy_36.txt) ---
Liczba pustych pól: 51
Starting SA: Initial Cost=47, T_init=1.00
  Iter: 2000, Temp: 0.8179, Cost: 30, Best: 17
  Iter: 4000, Temp: 0.6690, Cost: 22, Best: 11
  Iter: 6000, Temp: 0.5472, Cost: 16, Best: 6
  Iter: 8000, Temp: 0.4475, Cost: 5, Best: 2
  Iter: 10000, Temp: 0.3660, Cost: 5, Best: 2
  Iter: 12000, Temp: 0.2994, Cost: 2, Best: 2

Solution found at iteration 12164!
Finished SA: Total Iter=12164, Time=2.44s, Final T=0.2964, Best Cost=0

Najlepsze znalezione rozwiązanie:
--- Best Found (Cost=0) ---
5 3 4 | 6 7 8 | 9 1 2
6 7 2 | 1 9 5 | 3 4 